In [2]:
import itertools
import gc
from bib import *
import numpy as np
import pandas as pd
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras import backend as K 
from keras import  activations
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Dense, Activation, regularizers,BatchNormalization,Dropout
import tensorflow as tf
from  tqdm import tqdm_notebook
tf.logging.set_verbosity(tf.logging.ERROR)

Using TensorFlow backend.


In [3]:
lb = scipy.io.loadmat(dataPath +'VectLB19922008.mat')
labels = [k[0] for k in lb['labels'][0] ]
data=pd.DataFrame(lb['Vect'],columns=labels)
data['time']= data['year']*100+data['5days']
data=data.sort_values('time')
values=data.loc[:,['CHL '+ str(i) for i in range(2,19)]]


In [4]:
times=np.sort(list(set(data['time'])))

In [5]:
X=[]
y=[]
for i in range(len(times)-1):
    X.append(values.loc[data['time']==times[i],:].values)
    y.append(values.loc[data['time']==times[i+1],:].values)
X=np.array(X).reshape((-1,17))
y=np.array(y).reshape((-1,17))
# X=np.log(X)
# y=np.log(y)/np.log(10)

In [136]:
Xt=X[data['year'][:-9]!=2008]
Yt=y[data['year'][:-9]!=2008]

In [137]:
def generate(arch,act):
    layers=[]
    for index,size in enumerate(arch):
        if index ==0:
            layers.append(Dense(size, input_shape=(17,),activation=act[index]))
#             layers.append(BatchNormalization())

        else:
            layers.append(Dense(size,activation=act[index]))
#             layers.append(Dropout(0.3))
    layers.append(Dense(17,activation='linear',kernel_regularizer=regularizers.l2(0.01)))
    model = Sequential(layers)
    model.compile(optimizer='Adam',
              loss='mse')
    return model
callbacks = [EarlyStopping(monitor='val_loss', patience=15)]

In [138]:
def validate(arch,act,X,y):
    fold_evaluation=[]
    for k in tqdm_notebook(range(1992,2008),leave=False):
        center=np.logical_and(np.logical_and(np.logical_and(data['latitude']<=33, data['latitude']>=31 ), data['longitude']>=-65 ), data['longitude']<=-63 )
        xv=X[np.logical_and(data['year'][:-657]==k,center[:-657])]
        yv=y[np.logical_and(data['year'][:-657]==k,center[:-657])]
        xt=X[data['year'][:-657]!=k]
        yt=y[data['year'][:-657]!=k]
        
        model=generate(arch,act)
        mean=xt.mean()
        std=xt.std()
        xt=(xt-mean)/std
        xv=(xv-mean)/std
        
        model.fit(xt,yt,
          epochs=10000,
          callbacks=callbacks,
          verbose=0,
          batch_size=73*10, 
          validation_data=(xv,yv))
        yp = model.predict(xv)
        
#         yp=np.exp(yp*np.log(10))
#         yv=np.exp(yv*np.log(10))  
        fold_evaluation.append(evaluate(yp,yv))
        print(evaluate(yp,yv))
    return np.mean(fold_evaluation),np.std(fold_evaluation )
    

In [139]:
arch=(1000, 500, 1000, 1000, 1000, 2000)
K.clear_session()
print('ARCH :'+ str(arch))
# print('ACT :'+ str(act))
error,std=validate(arch,['softplus']*len(arch),Xt,Yt)
print('\t error: %.5f +- %.5f' % (error,std))

ARCH :(1000, 500, 1000, 1000, 1000, 2000)
0.09822339577323709
0.0878315320255171
0.08620101106937429
0.10935962372227462
0.096844111363202
0.0881275502091666
0.08337590261658655
0.08803039629329913
0.09209774298391891
0.09688925094847614
0.07479610010307873
0.07485244749058602
0.08721460060279106
0.09841504360655415
0.09169788441141498
0.07536467709407466
	 error: 0.08933 +- 0.00922


In [82]:
k=1995
arch=(1000, 1000, 2000)
K.clear_session()
xv=Xt[data['year'][:-657]==k]
yv=Yt[data['year'][:-657]==k]
xt=Xt[data['year'][:-657]!=k]
yt=Yt[data['year'][:-657]!=k]
model=generate(arch,['softplus']*len(arch))
mean=xt.mean()
std=xt.std()
xt=(xt-mean)/std
xv=(xv-mean)/std
model.fit(xt,yt,
  epochs=10000,
  callbacks=callbacks,
  verbose=1,
  batch_size=73*10, 
  validation_data=(xv,yv))

Train on 9855 samples, validate on 657 samples
Epoch 1/10000
9855/9855 [==============================] - 1s 77us/step - loss: 5.4135 - val_loss: 1.0076
Epoch 2/10000
9855/9855 [==============================] - 0s 18us/step - loss: 0.7922 - val_loss: 0.5383
Epoch 3/10000
9855/9855 [==============================] - 0s 18us/step - loss: 0.4492 - val_loss: 0.4009
Epoch 4/10000
9855/9855 [==============================] - 0s 17us/step - loss: 0.3690 - val_loss: 0.3631
Epoch 5/10000
9855/9855 [==============================] - 0s 17us/step - loss: 0.3482 - val_loss: 0.3534
Epoch 6/10000
9855/9855 [==============================] - 0s 17us/step - loss: 0.3400 - val_loss: 0.3463
Epoch 7/10000
9855/9855 [==============================] - 0s 17us/step - loss: 0.3345 - val_loss: 0.3416
Epoch 8/10000
9855/9855 [==============================] - 0s 19us/step - loss: 0.3293 - val_loss: 0.3354
Epoch 9/10000
9855/9855 [==============================] - 0s 17us/step - loss: 0.3233 - val_loss: 0.3277

In [93]:
xv=Xt[data['year'][:-657]==k].copy()
yv=Yt[data['year'][:-657]==k].copy()
xt=Xt[data['year'][:-657]!=k].copy()
yt=Yt[data['year'][:-657]!=k].copy()

mean=xt.mean()
std=xt.std()
xt=(xt-mean)/std
yp = model.predict(xv)

yp=np.exp(yp*np.log(10))
yv=np.exp(yv*np.log(10))  
evaluate(yp,yv)

ValueError: Tensor Tensor("dense_4/BiasAdd:0", shape=(?, 17), dtype=float32) is not an element of this graph.